In [38]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
from bs4 import BeautifulSoup
import re

In [18]:
df_ieee = pd.read_csv('../data/interim/ieee_author_df.csv')

In [19]:
# total number of authors in ieee before any manipulation/merging
df_ieee.shape

(12423, 9)

## IEEE Authors with more than one affiliation

In [20]:
# without one affiliation info
df_ieee[df_ieee['One Affiliation'].isnull()].shape

(167, 9)

In [21]:
# had more than one aff
df_ieee[(df_ieee['One Affiliation'].notnull()) & (df_ieee['One Affiliation'] != True)].shape

(337, 9)

In [22]:
df_ieee[(df_ieee['One Affiliation'].notnull()) & (df_ieee['One Affiliation'] != True)].shape[0] / df_ieee.shape[0]

0.02712710295419786

## Ieee authors missing affiliations info

In [23]:
df_ieee[df_ieee['Author Affiliation'].isnull()].shape

(167, 9)

In [24]:
df_ieee[df_ieee['Author Affiliation'].notnull()].shape[0]/df_ieee.shape[0]

0.9865571923045963

## Merged author

In [25]:
merged = pd.read_csv('../data/processed/merged_author_df.csv')

In [26]:
merged.shape

(12428, 27)

In [27]:
merged.head(1)

,Year,DOI,Title,IEEE Number of Authors,IEEE Author Position,IEEE Author Name,IEEE Author ID,IEEE Author Affiliation Updated,IEEE One Affiliation,Number of Authors,Author Name,Author Position,Author Position Type,OpenAlex Author ID,Author ORCID,Number of Affiliations,First Institution Name Updated,Raw Affiliation String Updated,First Institution ID,First Institution ROR,First Institution Type,First Institution Country Code,First Institution Country Code By Hand,First Institution Type By Hand,Binary Institution Type,Binary Institution Type By Hand,IEEE Author Affiliation Filled
0,2011,10.1109/TVCG.2011.185,D³ Data-Driven Documents,3.0,1.0,Michael Bostock,https://ieeexplore.ieee.org/author/37591067400,"Computer Science Department, Stanford University, Stanford, CA, USA",True,3.0,Michael Bostock,1.0,first,https://openalex.org/A2048345123,NaN,1.0,Stanford University,"Computer Science Department, Stanford University, Stanford, CA, USA#TAB#",https://openalex.org/I97018004,https://ror.org/00f54p054,education,US,NaN,NaN,education,NaN,"Computer Science Department, Stanford University, Stanford, CA, USA"


In [28]:
lack_type = merged[merged['First Institution Type'].isnull()].shape[0]
lack_type

2498

In [29]:
lack_cntry = merged[merged['First Institution Country Code'].isnull()].shape[0]
lack_cntry

2328

In [30]:
lack_type/merged.shape[0], lack_cntry/merged.shape[0]

(0.2009977470228516, 0.1873189571934342)

## Manually checking OpenAlex Author mapping accuracy

In [31]:
alex = pd.read_csv('../data/interim/openalex_author_df.csv')

In [32]:
alex.head(3)

,Year,DOI,Title,Number of Authors,Author Name,Author Position,Author Position Type,OpenAlex Author ID,Author ORCID,Number of Affiliations,First Institution Name,Raw Affiliation String,First Institution ID,First Institution ROR,First Institution Type,First Institution Country Code
0,2011,10.1109/TVCG.2011.185,D³ Data-Driven Documents,3,Michael Bostock,1,first,https://openalex.org/A2048345123,NaN,1.0,Stanford University,"Computer Science Department, Stanford University, Stanford, CA, USA#TAB#",https://openalex.org/I97018004,https://ror.org/00f54p054,education,US
1,2011,10.1109/TVCG.2011.185,D³ Data-Driven Documents,3,Vadim Ogievetsky,2,middle,https://openalex.org/A2668634103,NaN,1.0,Stanford University,"Computer Science Department, Stanford University, Stanford, CA, USA#TAB#",https://openalex.org/I97018004,https://ror.org/00f54p054,education,US
2,2011,10.1109/TVCG.2011.185,D³ Data-Driven Documents,3,Jeffrey Heer,3,last,https://openalex.org/A2112690490,https://orcid.org/0000-0002-6175-1655,1.0,Stanford University,"Computer Science Department, Stanford University, Stanford, CA, USA#TAB#",https://openalex.org/I97018004,https://ror.org/00f54p054,education,US


In [33]:
alex[(alex['Raw Affiliation String'].notnull()) & (
    alex['First Institution Type'].notnull()) & (
    alex['First Institution Country Code'].notnull())].sample(n = 100, random_state = 42)

,Year,DOI,Title,Number of Authors,Author Name,Author Position,Author Position Type,OpenAlex Author ID,Author ORCID,Number of Affiliations,First Institution Name,Raw Affiliation String,First Institution ID,First Institution ROR,First Institution Type,First Institution Country Code
4960,2010,10.1109/TVCG.2010.172,Interactive Visualization of Hyperspectral Images of Historical Documents,5,Michael S. Brown,5,last,https://openalex.org/A2594426199,NaN,1.0,National University of Singapore,"National University of singapore, singapore, Singapore",https://openalex.org/I165932596,https://ror.org/01tgyzw49,education,SG
3058,2011,10.1109/VAST.2011.6102439,Interactive visual comparison of multiple trees,6,Kay Hamacherk,6,last,https://openalex.org/A2327757246,NaN,1.0,TU Darmstadt,"Computational Biology, TU Darmstadt, Germany",https://openalex.org/I31512782,https://ror.org/05n911h24,education,DE
626,2016,10.1109/TVCG.2016.2598831,Towards Better Analysis of Deep Convolutional Neural Networks,6,Jun Zhu,5,middle,https://openalex.org/A3166362477,https://orcid.org/0000-0002-6254-2388,1.0,Tsinghua University,School of Software and TNListTsinghua University,https://openalex.org/I99065089,https://ror.org/03cve4549,education,CN
50,1998,10.1109/VISUAL.1998.745312,Simplifying surfaces with color and texture using quadric error metrics,2,Paul S. Heckbert,2,last,https://openalex.org/A350629882,NaN,1.0,Carnegie Mellon University,"Carnegie Mellon University, Pittsburgh, PA.",https://openalex.org/I74973139,https://ror.org/05x2bcf33,education,US
3270,2007,10.1109/TVCG.2007.70617,Visualization of Heterogeneous Data,7,Xin Dong,2,middle,https://openalex.org/A3181240397,NaN,1.0,University of Washington,"University of Washington , USA .",https://openalex.org/I201448701,https://ror.org/00cvxb145,education,US
12316,2021,10.1109/TVCG.2021.3114769,Differentiable Direct Volume Rendering,2,Rüdiger Westermann,2,last,https://openalex.org/A2030039896,https://orcid.org/0000-0002-3394-0731,1.0,Technical University of Munich,[Technical University of Munich (e-mail: westermann@tum.de)],https://openalex.org/I62916508,https://ror.org/02kkvpp62,education,DE
7901,2016,10.1109/TVCG.2016.2598826,PelVis: Atlas-based Surgical Planning for Oncological Pelvic Surgery,8,Elmar Eisemann,7,middle,https://openalex.org/A1986027970,https://orcid.org/0000-0003-4153-065X,1.0,Delft University of Technology,(Delft University of Technology),https://openalex.org/I98358874,https://ror.org/02e2c7k09,education,NL
5219,2009,10.1109/TVCG.2009.202,Visualizing the Intellectual Structure with Paper-Reference Matrices,3,Chaomei Chen,2,middle,https://openalex.org/A2147514866,https://orcid.org/0000-0001-8584-1041,1.0,Drexel University,"Drexel University, Philadelphia PA, USA#TAB#",https://openalex.org/I72816309,https://ror.org/04bdffz58,education,US
10034,2018,10.1109/TVCG.2018.2864847,Culling for Extreme-Scale Segmentation Volumes: A Hybrid Deterministic and Probabilistic Approach,6,Marco Agus,3,middle,https://openalex.org/A2150816462,https://orcid.org/0000-0003-2752-3525,1.0,King Abdullah University of Science and Technology,"King Abdullah University of Science and Technology (KAUST) , Saudi Arabia .",https://openalex.org/I71920554,https://ror.org/01q3tbs38,education,SA
620,2010,10.1109/TVCG.2010.183,OpinionSeer: Interactive Visualization of Hotel Customer Feedback,7,Hong Zhou,6,middle,https://openalex.org/A2423342565,NaN,1.0,Shenzhen University,"Shenzhen University, Shenzhen, China",https://openalex.org/I180726961,https://ror.org/01vy4gh70,education,CN


### Country:

- Bosch Research North America, should be 'US' but was labeled as 'DE'
- GMD German National Research Center for Information Technology, should be 'DE', but was labeled as 'US'

### Type
- European Centre for Medium-Range Weather Forecasts, Reading, UK, should be a nonprofit. Correct as binary
- GMD German National Research Center for Information Technology, is labeled as facility, but might be non-profit 
- Institute for High Energy Physics, Protvino, Moscow Region, is labeled as facility, but might be nonprofit

## Prediction accuracy

In [49]:
author = pd.read_csv('../data/ht_class/ht_cleaned_author_df.csv')

In [50]:
# correct. nothing is incomplete
author[(
    author['Affiliation Name'].isnull()) | (
    author['Affiliation Country Code'].isnull()) | (
    author['Affiliation Type'].isnull())]

,Year,DOI,Title,Number of Authors,Author Position,Author Name,OpenAlex Author ID,Affiliation Name,Affiliation Country Code,Affiliation Type,Binary Type,Cross-type Collaboration,International Collaboration


In [51]:
author.sample(100, random_state = 42)

,Year,DOI,Title,Number of Authors,Author Position,Author Name,OpenAlex Author ID,Affiliation Name,Affiliation Country Code,Affiliation Type,Binary Type,Cross-type Collaboration,International Collaboration
12277,2021,10.1109/tvcg.2021.3114817,An Efficient Dual-Hierarchy tSNE Minimization,3.0,2.0,Markus Billeter,https://openalex.org/A3203466205,"University of Leeds, England",GB,education,education,False,True
9621,2018,10.1109/TVCG.2018.2864887,InkPlanner: Supporting Prewriting via Intelligent Visual Diagramming,6.0,1.0,Zhicong Lu,https://openalex.org/A2611511841,"University of Toronto, Toronto, ON, CA",CA,education,education,True,True
11160,2020,10.1109/TVCG.2020.3030333,Interactive Visualization of Atmospheric Effects for Celestial Bodies,6.0,3.0,Carter Emmart,https://openalex.org/A2177988531,American Museum of Natural History,US,archive,non-education,True,True
12384,2021,10.1109/TVCG.2021.3114840,ThreadStates: State-based Visual Analysis of Disease Progression,5.0,2.0,Tali Mazor,https://openalex.org/A4263959575,"Dana-Farber Cancer Institute, USA",US,facility,non-education,True,True
12032,2021,10.1109/tvcg.2021.3057483,Net2Vis - A Visual Grammar for Automatically Generating Publication-Tailored CNN Architecture Visualizations,3.0,3.0,Timo Ropinski,https://openalex.org/A292280582,"Visual Computing Group at Ulm University, Ulm, Germany",DE,education,education,False,False
2439,2015,10.1109/TVCG.2015.2467619,Interactive Visual Discovering of Movement Patterns from Sparsely Sampled Geo-tagged Social Media Data,8.0,3.0,Zhenhuang Wang,https://openalex.org/A2345859439,"Key Laboratory of Machine Perception (Ministry of Education), School of EECS, Peking University",CN,education,education,False,True
5928,1992,10.1109/VISUAL.1992.235191,Optimizing triangulations by curvature equalization,2.0,1.0,L.L. Scarlatos,https://openalex.org/A16974148,"Grumman Data Systems, Woodbury, NY, USA",US,company,non-education,True,False
1114,2011,10.1109/TVCG.2011.181,Composite Density Maps for Multivariate Trajectories,6.0,6.0,Jarke J. van Wijk,https://openalex.org/A2317229737,"Eindhoven University of Technology, Netherlands",NL,education,education,True,True
12178,2021,10.1109/tvcg.2021.3114823,Visual Arrangements of Bar Charts Influence Comparisons in Viewer Takeaways,6.0,6.0,Steven Franconeri,https://openalex.org/A102385966,"Northwestern University, United States",US,education,education,True,True
8736,1996,10.1109/VISUAL.1996.568140,Data level comparative visualization in aircraft design,2.0,1.0,J. Trapp,https://openalex.org/A2149325771,"German AeroSpace Research Establishment, DLR, Guttingen, Germany",DE,facility,non-education,False,False


### Type

- JHU Applied Physics Lab, facility? Yes. At least I think it is not edu
- INRIA, gov -> facility
- Palo Alto research center, inc. facility -> com
- Databaselab, epfl, facility -> edu
- I.N.R.I.A. Sophia Antipolis, com -> facility
- I.N.R.I.A. Sophia Antipolis, gov -> facility


### Country

- JD Intelligent Cities Research, JD Tech, Beijing, China, should be CN but was labeled as GB


## Lowest concept scores

In [52]:
OPENALEX_CONCEPT_DF = pd.read_csv('../data/processed/openalex_concept_df.csv')

In [54]:
min(OPENALEX_CONCEPT_DF.Score)

0.3200384